<a href="https://colab.research.google.com/github/MariaBlancoGonzalez/Machine-Learning-22/blob/main/notebooks/NLP_Preprocessing_Vectorization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# NLP COMMENTS ASSIGNMENT

This work has been carried out by:


*   María Blanco González-Mohíno
*   Pablo Velasco Crespo
*   José Alberto Seco Sánchez-Camacho
*   Adrián Ruíz Esteban

# Data load

In [46]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.tokenize import word_tokenize
import pickle
from google.colab import files
import nltk
from nltk.stem import WordNetLemmatizer

Load data from github

In [47]:
!wget https://github.com/MariaBlancoGonzalez/Machine-Learning-22/raw/main/data/raw/comments.csv

--2022-01-21 19:38:01--  https://github.com/MariaBlancoGonzalez/Machine-Learning-22/raw/main/data/raw/comments.csv
Resolving github.com (github.com)... 140.82.113.3
Connecting to github.com (github.com)|140.82.113.3|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/MariaBlancoGonzalez/Machine-Learning-22/main/data/raw/comments.csv [following]
--2022-01-21 19:38:02--  https://raw.githubusercontent.com/MariaBlancoGonzalez/Machine-Learning-22/main/data/raw/comments.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.110.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1691506 (1.6M) [text/plain]
Saving to: ‘comments.csv.1’

comments.csv.1      100%[===================>]   1.61M  --.-KB/s    in 0.01s   

2022-01-21 19:38:02 (148 MB/s) - ‘comments.csv.1

In [48]:
comments = pd.read_csv('comments.csv')
comments.head(6)

,class,text
0,Auto,I have recently purchased a J30T with moderat...
1,Camera,I bought this product because I need instant ...
2,Auto,I have owned my Buick since 53000 km and I am...
3,Camera,This was my first Digital camera so I did qui...
4,Camera,Minolta DiMAGE 7Hi is in a digital SLR with 5...
5,Auto,I had a 1984 Saab a 1996 Saab and now a 1999...


# Preprocessing step

In [49]:
!pip install langdetect
from langdetect import detect
import re

In [50]:
text_list = comments['text'].tolist()
detect(text_list[0])

'en'

Language of the text: English.

Next steps will be develop taking into account the text has been writing in English.

## Remove useless data and all capital letters 


Here is were we extract the useless symbols and set all text to lower words

In [51]:
comments.shape

(600, 2)

In [52]:
useless_symbols = '!"_$%&/()=_ˆ*¡@'

In [53]:
comments['text_without_symbols'] = comments.apply(lambda row:
                                          re.sub(useless_symbols, '', row['text'], flags=re.ASCII), axis=1)
comments['text_without_symbols'] = comments.apply(lambda row: row['text_without_symbols'].lower(), axis=1)

This part is just to test if there are some useless symbols after removing it

In [54]:
x = comments.apply(lambda row: re.findall(u'!"_$%&/()=_ˆ*¡@', row['text_without_symbols']), axis = 1)

In [55]:
for i in x:
  if i != []:
    print(i)

No useless caracter found after this preprocessing step so this has been done correctly

In [56]:
comments.head(4)

,class,text,text_without_symbols
0,Auto,I have recently purchased a J30T with moderat...,i have recently purchased a j30t with moderat...
1,Camera,I bought this product because I need instant ...,i bought this product because i need instant ...
2,Auto,I have owned my Buick since 53000 km and I am...,i have owned my buick since 53000 km and i am...
3,Camera,This was my first Digital camera so I did qui...,this was my first digital camera so i did qui...


In [57]:
comments.shape

(600, 3)

## Remove Contractions

Those are all contractions that we are going to change



In [58]:
contractions = { 
"a'ight":"alright",
"ain't": "am not",
"amn't":"am not",
"aren't": "are not",
"can't": "cannot",
"can't've": "cannot have",
"'cause": "because",
"could've": "could have",
"couldn't": "could not",
"couldn't've": "could not have",
"didn't": "did not",
"doesn't": "does not",
"don't": "do not",
"hadn't": "had not",
"hadn't've": "had not have",
"hasn't": "has not",
"haven't": "have not",
"he'd": "he would",
"he'd've": "he would have",
"he'll": "he will",
"he'll've": "he will have",
"he's": "he is",
"how'll": "how will",
"I'd": "I would",
"I'd've": "I would have",
"I'll": "I will",
"I'll've": "I will have",
"I'm": "I am",
"I've": "I have",
"isn't": "is not",
"it'd": "it would",
"it'd've": "it would have",
"it'll": "it will",
"it'll've": "it will have",
"it's": "it is",
"let's": "let us",
"ma'am": "madam",
"mayn't": "may not",
"might've": "might have",
"mightn't": "might not",
"mightn't've": "might not have",
"must've": "must have",
"mustn't": "must not",
"mustn't've": "must not have",
"needn't": "need not",
"needn't've": "need not have",
"o'clock": "of the clock",
"oughtn't": "ought not",
"oughtn't've": "ought not have",
"shan't": "shall not",
"sha'n't": "shall not",
"she'd": "she would",
"she'd've": "she would have",
"she'll": "she will",
"she'll've": "she will have",
"she's": "she is",
"should've": "should have",
"shouldn't": "should not",
"shouldn't've": "should not have",
"so've": "so have",
"so's": "so as",
"that'd": "that would",
"that'd've": "that would have",
"that's": "that is",
"there'd": "there would",
"there'd've": "there would have",
"there's": "there is",
"they'd": "they would",
"they'd've": "they would have",
"they'll": "they will",
"they'll've": "they will have",
"they're": "they are",
"they've": "they have",
"to've": "to have",
"wasn't": "was not",
"we'd": "we would",
"we'd've": "we would have",
"we'll": "we will",
"we'll've": "we will have",
"we're": "we are",
"we've": "we have",
"weren't": "were not",
"what'll": "what will",
"what'll've": "what will have",
"what're": "what are",
"what's": "what is",
"what've": "what have",
"when's": "when is",
"when've": "when have",
"where's": "where is",
"where've": "where have",
"who'll": "who will",
"who'll've": "who will have",
"who's": "who is",
"who've": "who have",
"why's": "why is",
"why've": "why have",
"will've": "will have",
"won't": "will not",
"won't've": "will not have",
"would've": "would have",
"wouldn't": "would not",
"wouldn't've": "would not have",
"y'all": "you all",
"y'all'd": "you all would",
"y'all're": "you all are",
"y'all've": "you all have",
"you'd": "you would",
"you'd've": "you would have",
"you'll": "you will",
"you'll've": "you will have",
"you're": "you are",
"you've": "you have"
}

Method to replace all contractions

In [59]:
def replace_contractions(sentence):
  sentence= sentence.split(" ")
  for word in sentence:
    if word in contractions:
      sentence.replace(word, contractions.get(word))
  return ' '.join(sentence)

In [60]:
comments['text_no_contractions'] = comments.apply(lambda row: replace_contractions(row['text_without_symbols']), axis=1)
comments.head(4)

,class,text,text_without_symbols,text_no_contractions
0,Auto,I have recently purchased a J30T with moderat...,i have recently purchased a j30t with moderat...,i have recently purchased a j30t with moderat...
1,Camera,I bought this product because I need instant ...,i bought this product because i need instant ...,i bought this product because i need instant ...
2,Auto,I have owned my Buick since 53000 km and I am...,i have owned my buick since 53000 km and i am...,i have owned my buick since 53000 km and i am...
3,Camera,This was my first Digital camera so I did qui...,this was my first digital camera so i did qui...,this was my first digital camera so i did qui...


In [61]:
comments.shape

(600, 4)

## Spelling corrector

We implemented a spelling corrector but it takes too long to execute so it is commented.

In [62]:
'''
from textblob import TextBlob
text_array = []
for text in comments['text_no_contractions']:
  text_array.append(str(TextBlob(text).correct()))

comments['text_corrected'] = text_array
comments['text_corrected'].head(4)
'''

"\nfrom textblob import TextBlob\ntext_array = []\nfor text in comments['text_no_contractions']:\n  text_array.append(str(TextBlob(text).correct()))\n\ncomments['text_corrected'] = text_array\ncomments['text_corrected'].head(4)\n"

## Finding emojis

In [63]:
def emojis_founder(text):

  if (re.findall(u'\U0001F300-\U0001F5FF | \U0001F600-\U0001F64F | \U0001F680-\U0001F6FA | \u260E-\u26FF-\u2700-\u27BF', text)) != [] :
      print(re.findall(u'\U0001F300-\U0001F5FF | \U0001F600-\U0001F64F | \U0001F680-\U0001F6FA | \u260E-\u26FF-\u2700-\u27BF', text))

In [64]:
for i in comments['text_no_contractions'].tolist():
  emojis_founder(i)

No emojis found in the text, so we will not do anything about it

## Delete repeated words

In [65]:
def delete_repeated(text):
    text = text.split(" ")
    return " ".join([text[i] for i in range(len(text)) if i==0 or text[i]!=text[i-1]])

Here we delete the repeated words like 'great great ...'

In [66]:
comments['text_without_repeated_words'] = comments.apply(lambda row: delete_repeated(row['text_no_contractions']), axis=1)

In [67]:
comments['text_without_repeated_words'][1]

' i bought this product because i need instant gratification  i can t stand the torture of waiting for my photos to come back from the lab  and i don t want to pay extravagant sums for one hour developing  the sony mavica is the perfect solution for the click happy shutterbug who likes to take lots of photos  but doesn t have an in home darkroom or an interest in the nuances of traditional photography the great thing about the mavica is that you can take a photo and instantly see how it turned out on the built in color lcd display  if it sucked  take it again until you get it right  non digital cameras don t come close to offering this feature with film  you take the photo and cross your fingers  if it s a one time shot  forget it  when you re using the sony mavica  you can try a variety of shots  with or without flash  and look at the photos until you get what you want  you can take black and white photos or add a  quot sepia quot  effect that looks like an old tyme photo that you get

## Lemmatizer

In [68]:
nltk.download("popular")

[nltk_data] Downloading collection 'popular'
[nltk_data]    | 
[nltk_data]    | Downloading package cmudict to /root/nltk_data...
[nltk_data]    |   Package cmudict is already up-to-date!
[nltk_data]    | Downloading package gazetteers to /root/nltk_data...
[nltk_data]    |   Package gazetteers is already up-to-date!
[nltk_data]    | Downloading package genesis to /root/nltk_data...
[nltk_data]    |   Package genesis is already up-to-date!
[nltk_data]    | Downloading package gutenberg to /root/nltk_data...
[nltk_data]    |   Package gutenberg is already up-to-date!
[nltk_data]    | Downloading package inaugural to /root/nltk_data...
[nltk_data]    |   Package inaugural is already up-to-date!
[nltk_data]    | Downloading package movie_reviews to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Package movie_reviews is already up-to-date!
[nltk_data]    | Downloading package names to /root/nltk_data...
[nltk_data]    |   Package names is already up-to-date!
[nltk_data]    | Do

True

The method choose to compute the lemmatize part is the WordNetLemmatizer

In [69]:


lemmatizer = WordNetLemmatizer()

def lemmatize_text(text):
  # Set the text string to tokens with word_tokenize
    word_list = nltk.word_tokenize(text)
    return (' '.join([lemmatizer.lemmatize(w) for w in word_list]))

text_lemmatized = comments['text_without_repeated_words'].tolist()
text_lemma = []
for text in text_lemmatized:
  text_lemma.append(lemmatize_text(text))

comments['text_lemmatized'] = text_lemma
comments.head(4)

,class,text,text_without_symbols,text_no_contractions,text_without_repeated_words,text_lemmatized
0,Auto,I have recently purchased a J30T with moderat...,i have recently purchased a j30t with moderat...,i have recently purchased a j30t with moderat...,i have recently purchased a j30t with moderat...,i have recently purchased a j30t with moderate...
1,Camera,I bought this product because I need instant ...,i bought this product because i need instant ...,i bought this product because i need instant ...,i bought this product because i need instant ...,i bought this product because i need instant g...
2,Auto,I have owned my Buick since 53000 km and I am...,i have owned my buick since 53000 km and i am...,i have owned my buick since 53000 km and i am...,i have owned my buick since 53000 km and i am...,i have owned my buick since 53000 km and i am ...
3,Camera,This was my first Digital camera so I did qui...,this was my first digital camera so i did qui...,this was my first digital camera so i did qui...,this was my first digital camera so i did qui...,this wa my first digital camera so i did quite...


In [70]:
comments['text_lemmatized'][0]

'i have recently purchased a j30t with moderate mile i shopped for a car that wa looked after and paid more for it a a former auto technician i know what to look for and what to expect yet every time i drive this car i say to my self dam that s a great car and yes i truly do the car affords a level of style and comfort that is beyond compare for the price i had looked at a number of j30 s with some having covered 160k mile i checked them to see where the problem show they were great other than just basic owner neglect the car is extremely quiet and smooth yet doe not distance itself from a mild degree of sportiness performance is smooth yet impressive yet the power is well handled by braking and suspension system of similar refinement the appointment available with the car were somewhat sparse yet the standard of quality and functionality were apparent the bose sound system for example ha remarkable output and clarity without all the fancy dial my only complaint after 15k mile is a lac

In [71]:
comments['text_lemmatized'][1]

'i bought this product because i need instant gratification i can t stand the torture of waiting for my photo to come back from the lab and i don t want to pay extravagant sum for one hour developing the sony mavica is the perfect solution for the click happy shutterbug who like to take lot of photo but doesn t have an in home darkroom or an interest in the nuance of traditional photography the great thing about the mavica is that you can take a photo and instantly see how it turned out on the built in color lcd display if it sucked take it again until you get it right non digital camera don t come close to offering this feature with film you take the photo and cross your finger if it s a one time shot forget it when you re using the sony mavica you can try a variety of shot with or without flash and look at the photo until you get what you want you can take black and white photo or add a quot sepia quot effect that look like an old tyme photo that you get at the county fair a standard

## Store dataframe file after preprocessing

In [72]:
comments.to_pickle("comments.pkl")

# Vectorization

In [73]:
from sklearn.feature_extraction.text import TfidfVectorizer

## TFIDF

TF-IDF is a very common algorithm to transform text into a meaningful representation of numbers which is used to fit machine algorithm for prediction.  

Next, we are going to set the parameters for the vectorizer and then we transform and fit the column "text_lemmatized".

In [74]:
vectorizer = TfidfVectorizer(use_idf=True, smooth_idf=True)

data = vectorizer.fit_transform(comments['text_lemmatized'])
vectorizer.get_feature_names_out()

array(['00', '000', '01', ..., 'zooming', 'zx2', 'zx3'], dtype=object)

In [75]:
tfidf_df = pd.DataFrame(data.toarray(), columns=vectorizer.get_feature_names_out())
print(tfidf_df)

           00  000   01   02  024  ...  zoomed  zoomey  zooming  zx2  zx3
0    0.000000  0.0  0.0  0.0  0.0  ...     0.0     0.0      0.0  0.0  0.0
1    0.000000  0.0  0.0  0.0  0.0  ...     0.0     0.0      0.0  0.0  0.0
2    0.000000  0.0  0.0  0.0  0.0  ...     0.0     0.0      0.0  0.0  0.0
3    0.000000  0.0  0.0  0.0  0.0  ...     0.0     0.0      0.0  0.0  0.0
4    0.000000  0.0  0.0  0.0  0.0  ...     0.0     0.0      0.0  0.0  0.0
..        ...  ...  ...  ...  ...  ...     ...     ...      ...  ...  ...
595  0.000000  0.0  0.0  0.0  0.0  ...     0.0     0.0      0.0  0.0  0.0
596  0.065498  0.0  0.0  0.0  0.0  ...     0.0     0.0      0.0  0.0  0.0
597  0.000000  0.0  0.0  0.0  0.0  ...     0.0     0.0      0.0  0.0  0.0
598  0.000000  0.0  0.0  0.0  0.0  ...     0.0     0.0      0.0  0.0  0.0
599  0.000000  0.0  0.0  0.0  0.0  ...     0.0     0.0      0.0  0.0  0.0

[600 rows x 11736 columns]


In [76]:
import pickle

tfidf_df.to_pickle("df_tfidf.pkl")

## TFIDF+N-grams

In [77]:
# n-gramas:  ngram_range=(min,max)
vectorizer = TfidfVectorizer(ngram_range=(1,3))
data = vectorizer.fit_transform(comments['text_lemmatized'])

comments_df_ngrams =pd.DataFrame(data.toarray(),columns = vectorizer.get_feature_names_out())
comments_df_ngrams

,00,00 00,00 00 from,00 649,00 649 00,00 95,00 95 from,00 after,00 after bringing,00 and,00 and 100,00 and headed,00 and it,00 and wa,00 bottom,00 bottom line,00 but,00 but believe,00 but since,00 cad,00 cad the,00 camera,00 camera in,00 canon,00 canon powershot,00 clearance,00 clearance to,00 digital,00 digital camera,00 for,00 for it,00 for the,00 for this,00 from,00 from brandenburkhart,00 from www,00 give,00 give or,00 here,00 here my,...,zoomey camera cropping,zooming,zooming ability,zooming ability of,zooming after,zooming after you,zooming capability,zooming capability of,zooming etc,zooming etc very,zooming finder,zooming finder and,zooming flash,zooming flash head,zooming in,zooming in and,zooming in film,zooming in from,zooming in on,zooming in the,zooming need,zooming need not,zooming optical,zooming optical now,zooming ratio,zooming ratio with,zooming will,zooming will undermine,zx2,zx2 because,zx2 because of,zx2 bought,zx2 bought my,zx2 for,zx2 for single,zx2 new,zx2 new after,zx3,zx3 where,zx3 where lauded
0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
595,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
596,0.024677,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.037985,0.0,0.0,0.043997,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
597,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.

In [78]:
comments_df_ngrams.to_pickle("df_ngrams.pkl")

## TFIDF + N-grams + POS tagging

In [79]:
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

Each row is going to be tokenize, which means that each sentence is going to be a list of words. This is going to be useful while tagging.

In [80]:
# Word tokenization
comments['tokens'] = [word_tokenize(sentence) for sentence in comments['text_lemmatized']]
comments['tokens']

0      [i, have, recently, purchased, a, j30t, with, ...
1      [i, bought, this, product, because, i, need, i...
2      [i, have, owned, my, buick, since, 53000, km, ...
3      [this, wa, my, first, digital, camera, so, i, ...
4      [minolta, dimage, 7hi, is, in, a, digital, slr...
                             ...                        
595    [recently, our, 12, year, old, nissan, stanza,...
596    [i, always, do, a, lot, of, research, before, ...
597    [this, car, is, an, all, around, good, buy, if...
598    [i, waited, to, write, this, until, i, have, h...
599    [i, have, been, a, montero, owner, since, abou...
Name: tokens, Length: 600, dtype: object

Now we are going to tag each word, by tagging we refer to classify the words in verb, nouns...

In [81]:
tagged = comments['tokens'].apply(nltk.pos_tag)
tagged

0      [(i, NNS), (have, VBP), (recently, RB), (purch...
1      [(i, NN), (bought, VBD), (this, DT), (product,...
2      [(i, NNS), (have, VBP), (owned, VBN), (my, PRP...
3      [(this, DT), (wa, NN), (my, PRP$), (first, JJ)...
4      [(minolta, JJ), (dimage, NN), (7hi, CD), (is, ...
                             ...                        
595    [(recently, RB), (our, PRP$), (12, CD), (year,...
596    [(i, NN), (always, RB), (do, VBP), (a, DT), (l...
597    [(this, DT), (car, NN), (is, VBZ), (an, DT), (...
598    [(i, RB), (waited, VBD), (to, TO), (write, VB)...
599    [(i, NNS), (have, VBP), (been, VBN), (a, DT), ...
Name: tokens, Length: 600, dtype: object

In [82]:
pos_tag = []
sentence = ''
for comment in tagged:
  line = ''
  sentence = ''
  for word in comment:
    line = f'{word[0]}:{word[1]}'
    sentence += ' '+ ''.join(line)
  pos_tag.append(sentence)


In [83]:
pos_tag[3]

' this:DT wa:NN my:PRP$ first:JJ digital:JJ camera:NN so:IN i:JJ did:VBD quite:RB a:DT bit:NN of:IN research:NN unfortunately:RB pic:JJ quality:NN is:VBZ not:RB everything:NN a:DT i:NN found:VBD out:IN the:DT camera:NN is:VBZ really:RB cool:JJ and:CC take:VB great:JJ pic:NN but:CC the:DT viewer:NN is:VBZ a:DT bad:JJ place:NN by:IN that:DT i:JJ mean:VBP that:IN whenever:WRB i:JJ take:VBP a:DT picture:NN i:NN leave:VBP a:DT nose:JJ print:NN on:IN the:DT lcd:NN screen:NN it:PRP s:VBZ also:RB a:DT drag:NN to:TO have:VB to:TO slide:VB the:DT lens:NNS protector:VBP back:RB and:CC forth:NN to:TO turn:VB it:PRP on:IN and:CC off:IN one:CD other:JJ big:JJ time:NN bummer:NN is:VBZ that:IN it:PRP doesn:VBZ t:JJ come:NN with:IN an:DT ac:JJ adapter:NN fortunately:RB i:JJ have:VBP a:DT smartmedia:JJ reader:NN so:IN i:JJ don:VBP t:WDT have:VBP to:TO drain:VB my:PRP$ battery:NN while:IN downloading:VBG pic:VBP one:CD other:JJ thing:NN the:DT manual:NN is:VBZ a:DT damn:NN shame:NN it:PRP look:VBZ a:DT i

We attach the resulting row to comments

In [84]:
comments['pos_tag'] = pos_tag
comments

,class,text,text_without_symbols,text_no_contractions,text_without_repeated_words,text_lemmatized,tokens,pos_tag
0,Auto,I have recently purchased a J30T with moderat...,i have recently purchased a j30t with moderat...,i have recently purchased a j30t with moderat...,i have recently purchased a j30t with moderat...,i have recently purchased a j30t with moderate...,"[i, have, recently, purchased, a, j30t, with, ...",i:NNS have:VBP recently:RB purchased:VBN a:DT...
1,Camera,I bought this product because I need instant ...,i bought this product because i need instant ...,i bought this product because i need instant ...,i bought this product because i need instant ...,i bought this product because i need instant g...,"[i, bought, this, product, because, i, need, i...",i:NN bought:VBD this:DT product:NN because:IN...
2,Auto,I have owned my Buick since 53000 km and I am...,i have owned my buick since 53000 km and i am...,i have owned my buick since 53000 km and i am...,i have owned my buick since 53000 km and i am...,i have owned my buick since 53000 km and i am ...,"[i, have, owned, my, buick, since, 53000, km, ...",i:NNS have:VBP owned:VBN my:PRP$ buick:NN sin...
3,Camera,This was my first Digital camera so I did qui...,this was my first digital camera so i did qui...,this was my first digital camera so i did qui...,this was my first digital camera so i did qui...,this wa my first digital camera so i did quite...,"[this, wa, my, first, digital, camera, so, i, ...",this:DT wa:NN my:PRP$ first:JJ digital:JJ cam...
4,Camera,Minolta DiMAGE 7Hi is in a digital SLR with 5...,minolta dimage 7hi is in a digital slr with 5...,minolta dimage 7hi is in a digital slr with 5...,minolta dimage 7hi is in a digital slr with 5...,minolta dimage 7hi is in a digital slr with 5 ...,"[minolta, dimage, 7hi, is, in, a, digital, slr...",minolta:JJ dimage:NN 7hi:CD is:VBZ in:IN a:DT...
...,...,...,...,...,...,...,...,...
595,Auto,Recently our 12 year old Nissan Stanza decide...,recently our 12 year old nissan stanza decide...,recently our 12 year old nissan stanza decide...,recently our 12 year old nissan stanza decide...,recently our 12 year old nissan stanza decided...,"[recently, our, 12, year, old, nissan, stanza,...",recently:RB our:PRP$ 12:CD year:NN old:JJ nis...
596,Camera,I always do a lot of research before I buy an...,i always do a lot of research before i buy an...,i always do a lot of research before i buy an...,i always do a lot of research before i buy an...,i always do a lot of research before i buy any...,"[i, always, do, a, lot, of, research, before, ...",i:NN always:RB do:VBP a:DT lot:NN of:IN resea...
597,Auto,This car is an all around good buy If you ar...,this car is an all around good buy if you ar...,this car is an all around good buy if you ar...,this car is an all around good buy if you ar...,this car is an all around good buy if you are ...,"[this, car, is, an, all, around, good, buy, if...",this:DT car:NN is:VBZ an:DT all:RB around:RB ...
598,Auto,I waited to write this until I have had 4 mon...,i waited to write this until i have had 4 mon...,i waited to write this until i have had 4 mon...,i waited to write this until i have had 4 mon...,i waited to write this until i have had 4 mont...,"[i, waited, to, write, this, until, i, have, h...",i:RB waited:VBD to:TO write:VB this:DT until:...


In [85]:
vectorizer = TfidfVectorizer(ngram_range=(1,3))
data = vectorizer.fit_transform(comments['pos_tag'])

comments_df_ngrams_pos =pd.DataFrame(data.toarray(),columns = vectorizer.get_feature_names_out())
comments_df_ngrams_pos

,00,00 cd,00 cd 649,00 cd after,00 cd and,00 cd bottom,00 cd but,00 cd cad,00 cd camera,00 cd canon,00 cd cd,00 cd clearance,00 cd digital,00 cd dt,00 cd for,00 cd from,00 cd give,00 cd here,00 cd if,00 cd in,00 cd it,00 cd just,00 cd last,00 cd le,00 cd looking,00 cd making,00 cd new,00 cd nn,00 cd now,00 cd occasional,00 cd online,00 cd per,00 cd personal,00 cd photoshop,00 cd plus,00 cd pm,00 cd pny,00 cd sample,00 cd street,00 cd that,...,zoombrowser nnp,zoombrowser nnp ex,zoomed,zoomed nnp,zoomed nnp pixel,zoomed vbd,zoomed vbd into,zoomed vbn,zoomed vbn forward,zoomed vbn in,zoomed vbn out,zoomey,zoomey nn,zoomey nn camera,zooming,zooming nn,zooming nn ability,zooming nn after,zooming nn capability,zooming nn flash,zooming nn in,zooming nn will,zooming vbg,zooming vbg ability,zooming vbg etc,zooming vbg finder,zooming vbg in,zooming vbg need,zooming vbg optical,zooming vbg ratio,zx2,zx2 nn,zx2 nn because,zx2 nn for,zx2 nn nn,zx2 vbd,zx2 vbd new,zx3,zx3 nn,zx3 nn where
0,0.00000,0.00000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.00000,0.00000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.00000,0.00000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.00000,0.00000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.00000,0.00000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
595,0.00000,0.00000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
596,0.01926,0.01926,0.0,0.0,0.029646,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
597,0.00000,0.00000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0

In [86]:
comments_df_ngrams_pos.to_pickle("df_ngrams_pos.pkl")

## TFIDF + N-grams + POS tagging + other features

**Number of words**

In [87]:
words = 0
words = [words+len(comment) for comment in comments['pos_tag']]
sum(words)

2647254

**Number of sentences**

In [88]:
sentences = 0
sentences = [sentences+(len(comment.split('.'))) for comment in comments['pos_tag']]
sum(sentences)

600

**Number of verbs**

In [89]:
sentences = 0
sentences = [sentences+(len(comment.split('VB'))) for comment in comments['pos_tag']]
print("Verbs: ",sum(sentences))

Verbs:  54156


**Number of nouns**

In [90]:
sentences = 0
sentences = [sentences+(len(comment.split('NN'))) for comment in comments['pos_tag']]
print("Nouns: ", sum(sentences))

Nouns:  83857
